In [63]:
"""
NOTE: to run notebook use: venv/bin/python manage.py shell_plus --notebook
"""

MYPROJECT='/Users/erotundo/PycharmProjects/DatasetAnnotatorProj'
import os, sys
sys.path.insert(0, MYPROJECT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")

import django
django.setup()

from django.conf import settings
from DatasetAnnotator.models import Posts
from django.db.models import Q, F
from django.db.models import Count
from django.db.models.functions import Greatest, Coalesce
import io, json


# community selection
#available_dbs = settings.DATABASES.keys()[1:]
db = 'travel'
OUTPUT_PATH = 'AnalysisBestAnswer/output/' + db + '/data/'

# Extract threads to JSON file

In [64]:
# load all questions and answers
questions = Posts.objects\
        .using(db)\
        .filter(posttypeid=1)\
        .filter(acceptedanswerid__isnull=False)

all_answers = Posts.objects\
        .using(db)\
        .filter(posttypeid=2)

In [65]:
threads = []

print questions.count()

for question in questions:
    answers = all_answers.filter(parentid=question.id)
    
    q =  { 'title': question.title,
           'body': question.body,
           'user': question.owneruserid,
           'tags': question.tags,
    }
    
    a = []
    
    for answer in answers:
        if answer.id != question.acceptedanswerid:
            a.append({
               'body': answer.body,
               'user': answer.owneruserid,
            })
    accepted_ans = answers.get(id=question.acceptedanswerid)
    threads.append({
        'question': q,
        'other_answers': a,
        'accepted_answer': {
            'body': accepted_ans.body,
            'user': accepted_ans.owneruserid,
        }
    })

8027


In [66]:
print threads[0:1]

[{'accepted_answer': {'body': u'<p>from <a href="http://www.facebook.com/group.php?gid=16929680703Blockquote">here</a>:</p>&#xA;&#xA;<blockquote>&#xA;  <p>Go here: &#xA;  <a href="http://www.facebook.com/help/delete_account">http://www.facebook.com/help/delete_account</a></p>&#xA;  &#xA;  <p>Click "Submit" and follow the instructions.</p>&#xA;  &#xA;  <p>Your account will be deactivated for two weeks, and if you DO NOT USE FACEBOOK IN ANY WAY during that period, your account is permanently deleted. </p>&#xA;</blockquote>&#xA;', 'user': 22L}, 'question': {'body': u"<p>I don't trust Facebook's new privacy settings and have decided to delete my account, but I can't find the kill switch.</p>&#xA;&#xA;<p>Does anyone know how I can delete my Facebook account?</p>&#xA;", 'tags': u'<facebook><delete><account-management>', 'user': 17L, 'title': u'How do I delete my Facebook account?'}, 'other_answers': [{'body': u'<p>Go to "Your Account".  Go to "Deactivate Account".</p>&#xA;&#xA;<p>Facebook is

In [67]:
with io.open(OUTPUT_PATH + 'threads.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(threads, ensure_ascii=False))

In [68]:
import gc
del threads
del all_answers
del questions
gc.collect()

0

# Extract users data to JSON file

In [69]:
from django.forms.models import model_to_dict

users = Posts.objects\
        .using(db)\
        .filter(posttypeid__in=[1,2])\
        .values('owneruserid')\
        .annotate(posts=Count('owneruserid'))\
        .values('owneruserid', 'posts')

users_dict = dict()
foo_ = [users_dict.update({d['owneruserid'] : { u'nr_posts': d['posts'] }}) for d in users]

In [70]:
with io.open(OUTPUT_PATH + 'users.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(users_dict, ensure_ascii=False))

In [71]:
del users
del users_dict
gc.collect()

4